In [1]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [2]:
import pandas as pd
import requests as rq
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert
from shapely.geometry import Point
import overpy
import ast
from decimal import Decimal

from script import geolocaliser
from script import process_data
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

1. Création du fichier des communes cotières

In [3]:
# URL du fichier Excel
url_cotier = 'https://comersis.com/free-download.php?f=communes_littorales_2019.xlsx&d=littoral'

# Télécharger le fichier
response = rq.get(url_cotier)

# Lire le fichier Excel dans un DataFrame
df_cotieres = pd.read_excel(BytesIO(response.content))
df_cotieres = df_cotieres[df_cotieres['Région'] != "Départements d'Outre-Mer"]


print(df_cotieres.columns)

ConnectionError: HTTPSConnectionPool(host='comersis.com', port=443): Max retries exceeded with url: /free-download.php?f=communes_littorales_2019.xlsx&d=littoral (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000025AED4604A0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
# Filtrer sur le Motif du classement
df_cotieres = df_cotieres[df_cotieres['Motif du classement'] == "Commune riveraine de la mer ou d'un océan"]
# df_cotieres = df_cotieres[df_cotieres['Population'] > 500]

print(df_cotieres.columns)
liste_cotieres = df_cotieres['Nom commune'].unique().tolist()
print(liste_cotieres[1:10])
# Statistiques par "Région" pour les communes filtrées
stats_region_filtered = df_cotieres.groupby('Région').apply(
    lambda x: pd.Series({
        'nombre_communes': x['Nom commune'].nunique(),
        'population_totale': x['Population'].sum()
    })
).reset_index()

# Afficher les résultats
tableau_communes = stats_region_filtered.fillna("").style.hide(axis="index")

# Afficher le tableau sans NaN et sans index
tableau_communes

Index(['Région', 'ID département', 'Nom département', 'Code Insee comm.',
       'Nom commune', 'Classement', 'Motif du classement',
       'nom de l'entité géographique dont le littoral est protégé',
       'Population', 'commune déleguée'],
      dtype='object')
['Beaulieu-sur-Mer', 'Cagnes-sur-Mer', 'Cannes', "Cap-d'Ail", 'Eze', 'Mandelieu-la-Napoule', 'Menton', 'Nice', 'Roquebrune-Cap-Martin']
                       Région  nombre_communes  population_totale
0                    Bretagne              194             941329
1                       Corse               94             240732
2             Hauts de France               52             391192
3                   Normandie              202             710948
4          Nouvelle Aquitaine               81             510392
5                   Occitanie               27             237370
6            Pays-de-la-Loire               46             353413
7  Provence-Alpes-Côte d'Azur               53            2291162


C:\Users\flori\AppData\Local\Temp\ipykernel_30376\2888150090.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stats_region_filtered = df_cotieres.groupby('Région').apply(


In [ ]:
# Charger le fichier GeoJSON des communes de France depuis le lien GitHub
url_commune = 'https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/communes.geojson'
response_commune = rq.get(url_commune)
communes_geojson = response_commune.json()

# Charger le GeoJSON dans un GeoDataFrame
gdf_communes = gpd.GeoDataFrame.from_features(communes_geojson['features'])
gdf_communes_cotieres = gdf_communes[gdf_communes['nom'].isin(liste_cotieres)]
gdf_communes_cotieres = gdf_communes_cotieres.merge(df_cotieres[['Nom commune', 'Population']], 
                                                    left_on='nom', 
                                                    right_on='Nom commune'
                                                    ).drop(columns=['Nom commune'])


In [ ]:
# Calculer les centroïdes pour chaque commune
gdf_communes_cotieres['centroid'] = gdf_communes_cotieres.geometry.centroid
gdf_communes_cotieres['latitude_centre'] = gdf_communes_cotieres['centroid'].y
gdf_communes_cotieres['longitude_centre'] = gdf_communes_cotieres['centroid'].x
gdf_communes_cotieres = gdf_communes_cotieres.drop(columns=['centroid'])

liste_commune = gdf_communes_cotieres['nom'].to_list()

On utilise l'API Overpass pour successivement géolocaliser les mairies, les stations de transport et les plages de la commune.

In [12]:
# Appliquer la fonction sur chaque commune
api = overpy.Overpass()

gdf_communes_cotieres[['latitude_mairie', 'longitude_mairie']] = gdf_communes_cotieres.apply(
    lambda row: pd.Series(geolocaliser.get_townhall_coordinates(row['nom'], api)),
    axis=1
)

Dans le cas où Overpass ne permet pas de géolocaliser la mairie, on fait une recherche par mot-clé à partir de l'API Adresse de l'Institut National Géographique (ING)

In [13]:
# Liste des mots-clés pour localiser les mairies
mots_cles = [
    "Mairie+de", 
    "Hotel+de+ville+de", 
    "Bureau+du+Maire+de", 
    "Salle+des+Fêtes+de", 
    "Maison+Communale+de", 
    "Centre+Administratif+de", 
    "Hôtel+de+Ville", 
    "Bâtiment+Municipal+de", 
    "Mairie+municipale+de", 
    "Maison+des+Services+de"
]

# Compléter les coordonnées manquantes
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles, 
    colonne_latitude='latitude_mairie', 
    colonne_longitude='longitude_mairie'
)

Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Bonifacio
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Bonifacio
Aucune donnée trouvée pour l'adresse : Maison+des+Services+de+Bonifacio
Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Saint-Jacut-de-la-Mer
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Saint-Jacut-de-la-Mer
Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Ouessant
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Ouessant
Aucune donnée trouvée pour l'adresse : Maison+des+Services+de+Ouessant
Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Zuydcoote
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Zuydcoote
Aucune donnée trouvée pour l'adresse : Maison+des+Services+de+Zuydcoote
Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Yport
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Yport
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Château-d'Olonn

In [14]:
# Liste des mots-clés pour localiser les plages
mots_cles_plages = [ 
    "Plage+de",
    "Plage"
]

# Compléter les coordonnées manquantes pour les plages
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles_plages, 
    colonne_latitude='latitude_plage', 
    colonne_longitude='longitude_plage'
)

In [ ]:
# Pour les plages
api = overpy.Overpass()

gdf_communes_cotieres['beach_coordinates'] = gdf_communes_cotieres.apply(
    lambda row: geolocaliser.get_beach_coordinates(row['nom'], api),
    axis=1
)

In [ ]:
# Pour les stations de transport
api = overpy.Overpass()

gdf_communes_cotieres['station'] = gdf_communes_cotieres.apply(
    lambda row: geolocaliser.get_station_coordinates(row['nom'], api),
    axis=1
)

# Liste des mots-clés pour localiser les gares
mots_cles_gares = [ 
    "Gare+de"
]

# Compléter les coordonnées manquantes pour les pgares
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles_gares, 
    colonne_latitude='latitude_gare', 
    colonne_longitude='longitude_gare'
)

In [ ]:
# Utilisation de la fonction avec un GeoDataFrame
api = overpy.Overpass()

gdf_communes_cotieres[['latitude_port', 'longitude_port']] = gdf_communes_cotieres.apply(
    lambda row: pd.Series(geolocaliser.get_ports(row['nom'], api)),
    axis=1
)

# Liste des mots-clés pour localiser les plages
mots_cles_ports = [ 
    "Port+de",
]

# Compléter les coordonnées manquantes pour les plages
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles_ports, 
    colonne_latitude='latitude_port', 
    colonne_longitude='longitude_port'
)

In [23]:
# Application
gdf_communes_cotieres['station'] = gdf_communes_cotieres['station'].apply(process_data.fix_coordinates_format)
gdf_communes_cotieres['beach_coordinates'] = gdf_communes_cotieres['beach_coordinates'].apply(process_data.fix_coordinates_format)

In [24]:
gdf_communes_cotieres["beach_coordinates"] = gdf_communes_cotieres.apply(
    lambda row: row["beach_coordinates"] + [(row["latitude_plage"], row["longitude_plage"])] if row["latitude_plage"] is not None and row["longitude_plage"] is not None else row["beach_coordinates"],
    axis=1
)

In [25]:
# Ajout des coordonnées à beach_coordinates
gdf_communes_cotieres["station"] = gdf_communes_cotieres.apply(
    lambda row: row["station"] + [(row["latitude_gare"], row["longitude_gare"])] if row["latitude_gare"] is not None and row["longitude_gare"] is not None else row["beach_coordinates"],
    axis=1
)

In [26]:
gdf_communes_cotieres = gdf_communes_cotieres.drop(columns=['latitude_plage','longitude_plage','latitude_gare', 'longitude_gare','code'])

In [29]:
gdf_communes_cotieres.to_csv('data/communes_cotieres.csv',encoding="utf8",sep=";",index=False)

In [30]:
# URL du fichier GeoJSON des départements de France
url_departement = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"

# Récupérer le fichier GeoJSON des départements de France
response = rq.get(url_departement)
departements_geojson = response.json()

# Créer une liste pour stocker les noms des départements
departement_names = []

# Extraire les noms des départements depuis les propriétés de chaque feature
for feature in departements_geojson['features']:
    departement_name = feature['properties']['nom']
    departement_names.append(departement_name)

# Créer un DataFrame avec les noms des départements
df_departements = pd.DataFrame(departement_names, columns=['departementName'])
    
# Afficher le DataFrame
print(df_departements)

          departementName
0                   Aisne
1                    Aube
2                Calvados
3                  Cantal
4            Eure-et-Loir
..                    ...
91           Loir-et-Cher
92                Mayenne
93     Meurthe-et-Moselle
94            Deux-Sèvres
95  Territoire de Belfort

[96 rows x 1 columns]
